# Instalação

In [ ]:
#!pip install xcompact3d_toolbox

In [ ]:
#!pip install hvplot

In [ ]:
#!pip install datashader

# Computação e Visualização

Esse tutorial é uma introdução de como computar, selecionar dados e montar gráficos com objetos xarray oferecidos pelo xcompact3d-toolbox.

Comece por importar as bibliotecas:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import xcompact3d_toolbox as x3d

import warnings
warnings.filterwarnings('ignore')

## Por que xarray?

As estruturas de dados são providas por [xarray](http://docs.xarray.dev/en/stable/index.html), que atribui rótulos de dimensão, coordenadas e atributos sob NumPy arrays. E utiliza [dask](https://dask.org/) para computação paralela.
O objetivo é acelerar o desenvolvimento e customização do pós-processamento.

Adicionalmente, xcompact3d-toolbox provem funcionalidades para [DataArray](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataArray) e [Dataset](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataset).

Mais detalhes da biblioteca [xarray](http://docs.xarray.dev/en/stable/index.html) estão disponíveis em [Overview: Why xarray?](http://docs.xarray.dev/en/stable/getting-started-guide/why-xarray.html) e [Quick overview](http://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html).

## Exemplo - Escoamento ao redor de um cilindro

Se usará como base o caso do escoamento ao redor de um cilindro disponível [online](https://github.com/fschuch/xcompact3d_toolbox_data).

In [ ]:
dataset, prm = x3d.tutorial.open_dataset("cylinder", cache=True, cache_dir="./example/")

Primeiro, visualiza-se o `dataset`:

In [ ]:
dataset

O banco de dados está no formato do [xarray.Dataset](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) com as variáveis `u` (vetor de velocidade), `pp` (pressão) e `epsi` (que descreve a geométria), suas coordenadas (`x`, `y`, `t` e `i`) e alguns metadados, como a versão do Xcompact3d usado para rodar a simulação e a `url` onde se pode achar a simulação.

Cada variável e coordenada pode ser acessada com a notação de ponto (i.g., `snapshot.pp`, `snapshot.u`, `snapshot.x`) ou com colchetes (i.g., `snapshot["pp"]`, `snapshot["u"]`, `snapshot["x"]`).

Começando pelo `epsi`, que representa a geometria (1 dentro do cilindro e 0 fora), pode-se visualizá-lo com o método [plot](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.plot.html):

In [ ]:
dataset.epsi.plot();

No exemplo, o array bidimensional é plotado com `.plot()`, que chama automaticamente [xarray.plot.pcolormesh()](http://docs.xarray.dev/en/stable/generated/xarray.plot.pcolormesh.html#xarray.plot.pcolormesh).

As funcionalidades do xarray para construir gráficos utilizam como base a biblioteca [matplotlib](https://matplotlib.org/).

Para melhorar a figura, modificam-se os axis do gráfico com `x` para o axis `x`, a coordenada `y` para o axis `y`, e definindo o aspecto do gráfico para `equal`.

In [ ]:
ax = dataset.epsi.plot(x="x", y="y")
ax.axes.set_aspect("equal")

Para os campos de velocidade e pressão, se iguinora os valores dentro do cilindro. Isso pode ser feito atribuindo `np.NaN` para os valores de `u` e `pp` onde `epsi` é zero.

Para tal tarefa, pode-se usar o método [xarray.Dataset.where](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.where.html).

In [ ]:
for var in ["u", "pp"]:
    dataset[var] = dataset[var].where(dataset.epsi == 0, np.nan)

Nota-se a expressividade do xarray quando essa comparação é feita. Lembrando que `epsi` é 2D (x, y), `pp` é 3D (x, y, t) e `u` é 4D (i, x, y, t). Isso só é possível porque xarray transmite os valores de `epsi` em cada ponto da coordenada `t` e `i`.

Outra flexibilidade do xarray é que se pode selecionar dados baseados nas coordenadas e não apenas nos índices como numpy arrays.

Na célula a baixo, selecionou-se um ponto do domínio usando o método [select](http://docs.xarray.dev/en/stable/generated/xarray.Dataset.sel.html). Observa-se que se pode monitorar esse ponto durante toda a evolução da simulação:

In [ ]:
dataset.sel(x=10.0, y=6.0, method="nearest")

Pode-se combinar os métodos [sel](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.sel.htmlem) e [plot](https://docs.xarray.dev/en/stablegenerated/xarray.DataArray.plot.html) em apenas uma linha de código.

In [ ]:
dataset.u.sel(x=10.0, y=6.0, method="nearest").plot(x="t", hue="i");

Em outro exemplo, se plotou a [média](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.mean.html) temporal ($60 \le t \le 150$) da velocidade vertical, onde $x=10$:

In [ ]:
dataset.u.sel(x=10.0, t=slice(60.0, 150.0)).mean("t").plot(y="y", hue="i");

Pode-se observar que xarray permite que se referencie as coordenadas pelo seu nome ao invés do seu índice.

Estendendo esse conceito, se calculou a evolução temporal da energia cinética. Dada pela equação:

$$
k = \int_V \dfrac{u_iu_i}{2} dV.
$$

Com o código:

In [ ]:
dataset["kinetic_energy"] = ((dataset.u**2.0).sum("i").x3d.simpson("x", "y")) * 0.5
dataset["kinetic_energy"].attrs = {"name": "k", "long_name": "kinetic Energy", "units": "-"}
dataset["kinetic_energy"].plot();

No código acima:

* Foi calculada a equação para toda a série temporal com um código bem sucinto;

* Foram incluídos atributos para descrever o que foi computado. Essas informações foram automaticamente incluídas no plot;

* Fez-se o gráfico da energia cinética.

Pode-se obter as dimensões para o cálculo da integral volumétrica com:

In [ ]:
V_coords = [dim for dim in dataset.u.coords if dim in "xyz"]
V_coords

Reescrevendo o exemplo anterior, pode-se obter uma versão para o caso n-dimensional:

In [ ]:
dataset["kinetic_energy"] = ((dataset.u**2.0).sum("i").x3d.simpson(*V_coords)) * 0.5
dataset["kinetic_energy"].attrs = {"name": "k", "long_name": "kinetic Energy", "units": "-"}

Voltando ao plot 2D, para o campo de velocidade `u`, calculou-se a média temporal ([mean](https://docs.xarray.dev/en/stablegenerated/xarray.DataArray.mean.html)) para o intervalo de tempo $60 \le t \le 150$ ([sel](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.sel.html)) e montou-se o gráfico ([plot](https://docs.xarray.dev/en/stablegenerated/xarray.DataArray.plot.html)):

In [ ]:
g = dataset.u.sel(t=slice(60.0, 150.0)).mean("t").plot(x="x", y="y", row="i", cmap="turbo", rasterized=True)
for ax in g.axes.flat:
    ax.axes.set_aspect("equal")

Para ver a evolução temporal de `u` pode-se usar o método [isel](https://docs.xarray.dev/en/stablegenerated/xarray.DataArray.isel.html) e a função [slice](https://docs.python.org/3/library/functions.html#slice) para selecionar amostras a cada 40 intervalos de tempo (do contrário haveria figuras demais) e [plot](https://docs.xarray.dev/en/stablegenerated/xarray.DataArray.plot.html):

In [ ]:
g = dataset.u.isel(t=slice(None, None, 40)).plot(x="x", y="y", col="t", row="i", cmap="turbo", rasterized=True)

for ax in g.axes.flat:
    ax.axes.set_aspect("equal")

Como exemplo de computação paralela, calcula-se a vorticidade para o banco de dados. Para uma configuração bidimensional, a equação da vorticidade pode ser escrita da forma:

$$
\omega_z = \dfrac{\partial u_y}{\partial x}  - \dfrac{\partial u_x}{\partial y}.
$$

Pode-se usar [xarray.DataArray.differentiate](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.differentiate.html) com uma derivada centrada de 2th ordem. Todavia, há opção de usar um esquema centrado de 4th ordem disponível em [X3dDataArray.first_derivative](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataArray.first_derivative).

Primeiro se define o atributo da condição de contorno para o campo de velocidade:

In [ ]:
dataset["u"].attrs["BC"] = prm.get_boundary_condition("u")

e calcular a vorticidade:

In [ ]:
%%time
dataset["vort"] = dataset.u.sel(i="y").x3d.first_derivative("x") - dataset.u.sel(i="x").x3d.first_derivative("y")

CPU times: user 1.3 s, sys: 32.8 ms, total: 1.33 s
Wall time: 1.35 s


Lembrando que a equação acima calcula a vorticidade para toda a série temporal.

O método [X3dDataArray.pencil_decomp](https://docs.fschuch.com/xcompact3d_toolbox/references/api-reference.html#xcompact3d_toolbox.array.X3dDataArray.pencil_decomp) pode ser usado para converter o campo de velocidade para um dask array\ para computação paralela (veja [Using Dask with xarray](http://docs.xarray.dev/en/stable/user-guide/dask.html#using-dask-with-xarray)).

In [ ]:
u_chunked = dataset.u.x3d.pencil_decomp("x", "y")
u_chunked

Neste caso, `pencil_decomp` agrupou todos os dados em apenas um `chunk`. Isto ocorre porque, por padrão, `pencil_decomp` procura a decomposição que otimize o processamento em paralelo. Nesse caso, o banco de bases é muito pequeno que não vale a pena usar paralelismo.

Porém, pode-se definir manualmente o número de `chunks`. Por exemplo, pode-se dividir o domínio t em 4 pencils que podem ser resolvidos por 4 núcleos em paralelo.

In [ ]:
u_chunked = dataset.u.chunk(chunks={"t": 51})
u_chunked

Calculando a vorticidade em paralelo:

In [ ]:
%%time
dataset["vort"] = (
    u_chunked.sel(i="y").x3d.first_derivative("x") - u_chunked.sel(i="x").x3d.first_derivative("y")
).compute()

CPU times: user 3.04 s, sys: 180 ms, total: 3.22 s
Wall time: 3.58 s


Como se observa, não há uma melhora de desempenho por ser um banco de dados muito pequeno, o que não compensa a sobrecarga para utilizar o paralelismo. Porém, pode-se utilizar esse código para escalar para simulações maiores.

Novamente, pode-se definir atributos para a grandeza computada.

In [ ]:
dataset["vort"].attrs = {"name": "wz", "long_name": "Vorticity", "units": "-"}

E a partir dela gerar o gráfico:

In [ ]:
g = dataset.vort.isel(t=slice(None, None, 10)).plot(
    x="x", y="y", col="t", col_wrap=7, cmap="turbo", rasterized=True, robust=True
)
for ax in g.axes.flat:
    ax.axes.set_aspect("equal")

A precisão próxima à geometria pode ser melhorada interpolando o campo dentro do cilindro, dessa forma ter-se-ia uma função contínua para aplicar-se a derivada. Para visualização, pode-se selecionar uma amostra dos dados para que se consiga visualizar em 1D. Para o campo de velocidade, selecionou-se apenas a componente `x`.

In [ ]:
ux_sample = dataset.u.sel(i="x", t=150.0, y=6.0)

Pode-se montar o gráfico para essa amostra com `np.NaN` dentro do cilindro e preenchê-la com [uma interpolação cúbica](http://docs.xarray.dev/en/stable/generated/xarray.DataArray.interpolate_na.html#xarray.DataArray.interpolate_na).

In [ ]:
ux_sample.plot(label="NaN at the Geometry")
ux_sample.interpolate_na("x", "cubic").plot(label="Interpolated", zorder=-1)
plt.legend();

Comparando a derivada com e sem a interpolação.

In [ ]:
ux_sample.x3d.first_derivative("x").plot(label="NaN at the Geometry")
ux_sample.interpolate_na("x", "cubic").x3d.first_derivative("x").plot(label="Interpolated", zorder=-1)
plt.ylabel("du/dx")
plt.legend();

[xarray](http://docs.xarray.dev/en/stable/) é construído sobre [Numpy](https://numpy.org/), seus arrays e datasets são compatíveis com várias ferramentas Numpy/SciPy.
Você pode acessar `numpy.ndarray` objetos com a propriedade `values`:

In [ ]:
dataset.epsi.values

Veja também:

* [Xarray: How do I ...](http://docs.xarray.dev/en/stable/howdoi.html)
* [Xarray's tutorials](https://xarray-contrib.github.io/xarray-tutorial/)
* [python-xarray](https://stackoverflow.com/questions/tagged/python-xarray) no StackOverflow
* [pint-xarray](https://pint-xarray.readthedocs.io/en/latest/) Pint permite converter unidades em Python

### Visualização interativa

In [ ]:
import hvplot.xarray  # noqa: F401

Todos os exemplos anteriores foram baseados em matplotlib, porém xarray é compatível com mais opções. Um deles é [hvPlot](https://hvplot.holoviz.org/index.html) (veja [Gridded Data](https://hvplot.holoviz.org/user_guide/Gridded_Data.html)).

hvPlot é mais recomendado quando se está explorando seus dados e precisa-se de maior interatividade.

Como exemplo, vamos refazer um dos gráficos anteriores, escolhendo um ponto no domínio e visualizando a evolução da velocidade:

In [ ]:
dataset.u.sel(x=10.0, y=6.0, method="nearest").hvplot(x="t", by="i")

Uma das funcionalidades do hvPlot é quando se passa mais coordenadas do que ele pode plotar, as coordenadas extras podem ser manejadas com widgets. Então, se um ponto não foi selecionado previamente, pode-se usar widgets para selecionar o ponto no domínio.

In [ ]:
dataset.u.hvplot(x="t", by="i", widget_location="bottom")

Aqui se montou o gráfico para a energia cinética, agora com hvPlot.

In [ ]:
dataset["kinetic_energy"].hvplot()

Por último, pode-se ver uma animação com o campo de velocidade gerada com poucas linhas de código.

In [ ]:
dataset.vort.sel(t=slice(40, None)).hvplot(
    x="x",
    y="y",
    aspect="equal",
    clim=(-5, 5),
    rasterize=True,
    cmap="turbo",
    widget_type="scrubber",
    widget_location="bottom",
    title="Escoamento ao redor de um cilindro",
    clabel="Vorticidade [-]",
)